### libraries

In [55]:
import pandas as pd
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

### 시뮬레이션 파라미터

In [56]:
# 주요 암 유전자의 실제 좌표 사용
target_regions = {
    'TP53': {'chr': 'chr17', 'start': 7571720, 'end': 7590868, 'positions_per_gene': 2500},
    'EGFR': {'chr': 'chr7', 'start': 55086714, 'end': 55275031, 'positions_per_gene': 2000},
    'KRAS': {'chr': 'chr12', 'start': 25357723, 'end': 25403870, 'positions_per_gene': 1500}
}

# 시뮬레이션 파라미터
umi_mean = 500  # 평균 UMI 분자 수
umi_std = 100
avg_pcr_copies = 100  # PCR 평균 증폭 횟수
pcr_std = 30

In [57]:
target_regions

{'TP53': {'chr': 'chr17',
  'start': 7571720,
  'end': 7590868,
  'positions_per_gene': 2500},
 'EGFR': {'chr': 'chr7',
  'start': 55086714,
  'end': 55275031,
  'positions_per_gene': 2000},
 'KRAS': {'chr': 'chr12',
  'start': 25357723,
  'end': 25403870,
  'positions_per_gene': 1500}}

In [58]:
# VAF 분포 정의
tumor_fractions = {
    'high': 0.05,      # 5% - 명확한 변이
    'medium': 0.015,   # 1.5% - 감지 가능
    'low': 0.005       # 0.5% - 경계 케이스
}

sequencing_error_rate = 0.0005  # Sequencing error rate
umi_error_rate = 0.0001  # UMI error rate

In [59]:
tumor_fractions

{'high': 0.05, 'medium': 0.015, 'low': 0.005}

In [60]:
# Hotspot 변이 정의 (유전자별)
hotspot_variants = {
    'TP53': [
        {'pos': 7577548, 'vaf': 'high', 'variant': 'R248Q'},
        {'pos': 7577559, 'vaf': 'high', 'variant': 'R248W'},
        {'pos': 7578190, 'vaf': 'medium', 'variant': 'R175H'},
        {'pos': 7578191, 'vaf': 'low', 'variant': 'Y234H'},
    ],
    'EGFR': [
        {'pos': 55191822, 'vaf': 'high', 'variant': 'L858R'},
        {'pos': 55191817, 'vaf': 'medium', 'variant': 'L861Q'},
        {'pos': 55086776, 'vaf': 'medium', 'variant': 'E746-A750'},
    ],
    'KRAS': [
        {'pos': 25398284, 'vaf': 'high', 'variant': 'G12C'},
        {'pos': 25398285, 'vaf': 'high', 'variant': 'G12V'},
        {'pos': 25398286, 'vaf': 'medium', 'variant': 'G13D'},
        {'pos': 25380275, 'vaf': 'low', 'variant': 'Q61H'},
    ]
}


In [61]:
hotspot_variants

{'TP53': [{'pos': 7577548, 'vaf': 'high', 'variant': 'R248Q'},
  {'pos': 7577559, 'vaf': 'high', 'variant': 'R248W'},
  {'pos': 7578190, 'vaf': 'medium', 'variant': 'R175H'},
  {'pos': 7578191, 'vaf': 'low', 'variant': 'Y234H'}],
 'EGFR': [{'pos': 55191822, 'vaf': 'high', 'variant': 'L858R'},
  {'pos': 55191817, 'vaf': 'medium', 'variant': 'L861Q'},
  {'pos': 55086776, 'vaf': 'medium', 'variant': 'E746-A750'}],
 'KRAS': [{'pos': 25398284, 'vaf': 'high', 'variant': 'G12C'},
  {'pos': 25398285, 'vaf': 'high', 'variant': 'G12V'},
  {'pos': 25398286, 'vaf': 'medium', 'variant': 'G13D'},
  {'pos': 25380275, 'vaf': 'low', 'variant': 'Q61H'}]}

### Functions

In [62]:
# ============================================================================
# 함수 정의
# ============================================================================

def generate_umi_count(mean=umi_mean, std=umi_std):
    """UMI 분자 수 생성 (정규분포)"""
    return max(int(np.random.normal(mean, std)), 10)

## NGS 변이 콜링(Variant Calling)**의 가장 기본이 되는 통계적 검증 방법 중 하나인 
## 이항분포 검정(Binomial Test)을 사용하여 p-value를 계산하는 함수입니다.
def binomial_test_pvalue(alt_reads, total_reads, error_rate):
    """이항분포 검정으로 p-value 계산"""
    if total_reads < 2:
        return 1.0
    return 1 - stats.binom.cdf(alt_reads - 1, total_reads, error_rate)

In [63]:
## Function to generate molecule counts based on UMI counts and true VAF
import numpy as np
# 이전 파라미터 및 generate_umi_count, avg_pcr_copies, pcr_std 등이 정의되어 있어야 함

def _generate_molecule_counts(n_unique_molecules, true_vaf):
    """UMI 분자 수와 true_vaf를 기반으로 변이/정상 분자 수를 결정"""
    # 이항 분포를 사용하여 변이 분자 수를 결정합니다.
    # N=n_unique_molecules, P=true_vaf
    n_variant_molecules = np.random.binomial(n_unique_molecules, true_vaf)
    n_ref_molecules = n_unique_molecules - n_variant_molecules
    return n_variant_molecules, n_ref_molecules

def _simulate_pcr_amplification(n_molecules):
    """주어진 분자 수에 대해 PCR 증폭 후 총 리드 수를 반환"""
    total_reads = 0
    for _ in range(n_molecules):
        # 정규분포를 사용하여 PCR 증폭 횟수(copies) 결정
        copies = max(1, int(np.random.normal(avg_pcr_copies, pcr_std)))
        total_reads += copies
    return total_reads

In [64]:
def simulate_gene_variants(gene_name, region, n_positions):
    """
    특정 유전자의 변이 시뮬레이션을 실행하고, 각 포지션에 대해 분석 및 필터링을 적용합니다.
    """
    hotspots_dict = {v['pos']: v for v in hotspot_variants.get(gene_name, [])}
    hotspot_positions = set(hotspots_dict.keys()) # 1. 핫스팟 위치 추출
    
    # 2. 배경 포지션 생성 (핫스팟 위치 제외)
    all_uniform_positions = set(np.linspace(region['start'], region['end'], n_positions, dtype=int))
    background_positions = list(all_uniform_positions - hotspot_positions)
    
    # 3. 핫스팟 위치와 배경 위치 통합
    simulation_positions = list(hotspot_positions) + background_positions
    
    data = []
    
    # 이제 simulation_positions에는 핫스팟이 반드시 포함됨
    for pos in simulation_positions:
        # 이전에 정의된 generate_variant_data를 사용하여 데이터 생성
        data_record = generate_variant_data(pos, gene_name, region, hotspots_dict)
        
        # 이전에 정의된 analyze_and_filter_variant를 사용하여 분석
        final_record = analyze_and_filter_variant(data_record)
        
        data.append(final_record)
    
    return data

In [65]:
# Function to generate variant data for a single genomic position

# 이전 파라미터, generate_umi_count, tumor_fractions, sequencing_error_rate 등이 정의되어 있어야 함

def generate_variant_data(pos, gene_name, region, hotspots):
    """
    단일 유전체 위치에 대한 시뮬레이션 데이터를 생성하고 기본 정보를 반환
    (통계 분석 및 필터링은 포함하지 않음)
    """
    hotspots = {v['pos']: v for v in hotspot_variants.get(gene_name, [])}
    n_unique_molecules = generate_umi_count()
    
    # 변이 정보 초기화
    n_variant_molecules, n_ref_molecules = 0, n_unique_molecules
    variant_reads, ref_reads, total_depth = 0, 0, 0
    is_true_variant = False
    variant_type = 'background'
    true_vaf = None

    if pos in hotspots:
        # ===== 🎯 진짜 변이 (Hotspot) 시뮬레이션 =====
        hotspot_info = hotspots[pos]
        true_vaf = tumor_fractions[hotspot_info['vaf']]
        
        # UMI 수준 변이 결정
        n_variant_molecules, n_ref_molecules = _generate_molecule_counts(n_unique_molecules, true_vaf)
        
        # PCR 증폭 시뮬레이션
        variant_reads = _simulate_pcr_amplification(n_variant_molecules)
        ref_reads = _simulate_pcr_amplification(n_ref_molecules)
        total_depth = variant_reads + ref_reads
        
        is_true_variant = True
        variant_type = hotspot_info['variant']
        
    else:
        # ===== 👻 배경 잡음 시뮬레이션 (Sequencing Error) =====
        # 모든 분자는 정상 (n_ref_molecules = n_unique_molecules)
        
        # 총 리드 수 (모든 정상 분자의 PCR 증폭)
        total_depth = _simulate_pcr_amplification(n_unique_molecules)
        
        # 시퀀싱 오류로 인한 변이 리드 발생
        # N=total_depth, P=sequencing_error_rate
        variant_reads = np.random.binomial(total_depth, sequencing_error_rate)
        ref_reads = total_depth - variant_reads
        
        # true_vaf는 0이지만 None으로 표시하여 배경 잡음임을 명확히 함
        
    
    vaf = variant_reads / total_depth if total_depth > 0 else 0
    
    return {
        'chrom': region['chr'],
        'position': pos,
        'gene': gene_name,
        'variant_type': variant_type,
        'is_true_variant': is_true_variant,
        'true_vaf': true_vaf,
        
        'n_unique_molecules': n_unique_molecules,
        'n_alt_molecules': n_variant_molecules,
        'n_ref_molecules': n_ref_molecules,
        
        'alt_reads': variant_reads,
        'ref_reads': ref_reads,
        'total_reads': total_depth,
        'vaf': vaf,
        'vaf_percent': vaf * 100
    }

In [66]:
# Function to analyze and filter variant data based on statistical tests

# 이전 파라미터, binomial_test_pvalue, sequencing_error_rate 등이 정의되어 있어야 함

def analyze_and_filter_variant(data_record):
    """생성된 데이터 레코드에 통계 검정 및 필터 기준을 적용"""
    
    # 필요한 값 추출
    variant_reads = data_record['alt_reads']
    total_depth = data_record['total_reads']
    vaf = data_record['vaf']
    n_variant_molecules = data_record['n_alt_molecules']
    
    # 통계 검정 (2단계에서 분리된 함수 사용)
    binomial_pval = binomial_test_pvalue(
        variant_reads, 
        total_reads=total_depth, 
        error_rate=sequencing_error_rate
    )
    
    # 로그 P-value 계산
    log10_pval = np.log10(binomial_pval) if binomial_pval > 0 else -300
    
    # ===== 📏 필터링 로직 정의 및 적용 =====
    
    # Stringent (엄격한) 필터 기준
    pass_stringent = (
        variant_reads >= 5 and
        n_variant_molecules >= 3 and
        vaf >= 0.005 and
        binomial_pval < 0.01 and
        total_depth >= 10000
    )
    
    # Sensitive (민감한) 필터 기준
    pass_sensitive = (
        variant_reads >= 3 and
        n_variant_molecules >= 2 and
        vaf >= 0.001 and
        binomial_pval < 0.05 and
        total_depth >= 5000
    )
    
    # 결과 레코드 업데이트
    data_record.update({
        'binomial_pval': binomial_pval,
        'log10_pval': log10_pval,
        'pass_stringent': pass_stringent,
        'pass_sensitive': pass_sensitive,
        
        # 개별 필터 상태 추가 (원래 코드와 동일)
        'pass_depth_stringent': total_depth >= 10000,
        'pass_depth_sensitive': total_depth >= 5000,
        'pass_alt_reads_stringent': variant_reads >= 5,
        'pass_alt_reads_sensitive': variant_reads >= 3,
        'pass_vaf_stringent': vaf >= 0.005,
        'pass_vaf_sensitive': vaf >= 0.001,
        'pass_pval_stringent': binomial_pval < 0.01,
        'pass_pval_sensitive': binomial_pval < 0.05,
        'pass_umi_stringent': n_variant_molecules >= 3,
        'pass_umi_sensitive': n_variant_molecules >= 2,
    })
    
    return data_record

### 메인 시뮬레이션

In [67]:
print("=" * 80)
print("ctDNA 변이콜링 시뮬레이션 시작")
print("=" * 80)

all_data = []

for gene_name, region in target_regions.items():
    print(f"\n[{gene_name}] 시뮬레이션 중...")
    gene_data = simulate_gene_variants(
        gene_name, 
        region, 
        region['positions_per_gene']
    )
    all_data.extend(gene_data)
    print(f"  - {len(gene_data)} 포지션 생성 완료")

sim_ctdna = pd.DataFrame(all_data)

ctDNA 변이콜링 시뮬레이션 시작

[TP53] 시뮬레이션 중...
  - 2504 포지션 생성 완료

[EGFR] 시뮬레이션 중...
  - 2003 포지션 생성 완료

[KRAS] 시뮬레이션 중...
  - 1504 포지션 생성 완료


### 결과 분석 및 출력

In [68]:
print("\n" + "=" * 80)
print("시뮬레이션 요약")
print("=" * 80)

print(f"\n📊 데이터셋 규모:")
print(f"  - 전체 포지션: {len(sim_ctdna):,}")
print(f"  - TP53: {len(sim_ctdna[sim_ctdna['gene']=='TP53']):,}")
print(f"  - EGFR: {len(sim_ctdna[sim_ctdna['gene']=='EGFR']):,}")
print(f"  - KRAS: {len(sim_ctdna[sim_ctdna['gene']=='KRAS']):,}")

print(f"\n📈 시퀀싱 통계:")
print(f"  - 평균 깊이: {sim_ctdna['total_reads'].mean():,.0f}")
print(f"  - 깊이 범위: {sim_ctdna['total_reads'].min():,} - {sim_ctdna['total_reads'].max():,}")
print(f"  - 평균 UMI 분자: {sim_ctdna['n_unique_molecules'].mean():.1f}")

print(f"\n🔍 변이 정보:")
print(f"  - 진정 변이 (True variant): {sim_ctdna['is_true_variant'].sum()}")
print(f"  - 배경 노이즈: {(~sim_ctdna['is_true_variant']).sum()}")

# 두 가지 필터 결과 비교
print(f"\n✅ 필터링 결과 비교:")
print(f"\n  [Stringent Filter]")
stringent_total = sim_ctdna['pass_stringent'].sum()
print(f"  - 통과: {stringent_total} ({stringent_total/len(sim_ctdna)*100:.2f}%)")

print(f"\n  [Sensitive Filter]")
sensitive_total = sim_ctdna['pass_sensitive'].sum()
print(f"  - 통과: {sensitive_total} ({sensitive_total/len(sim_ctdna)*100:.2f}%)")

print(f"\n📋 진정 변이 상세:")
true_vars = sim_ctdna[sim_ctdna['is_true_variant']].sort_values('vaf_percent', ascending=False)
print(true_vars[['gene', 'position', 'variant_type', 'n_alt_molecules', 'alt_reads', 
                 'total_reads', 'vaf_percent', 'pass_stringent', 'pass_sensitive']].to_string(index=False))



시뮬레이션 요약

📊 데이터셋 규모:
  - 전체 포지션: 6,011
  - TP53: 2,504
  - EGFR: 2,003
  - KRAS: 1,504

📈 시퀀싱 통계:
  - 평균 깊이: 49,443
  - 깊이 범위: 10,034 - 88,812
  - 평균 UMI 분자: 497.0

🔍 변이 정보:
  - 진정 변이 (True variant): 11
  - 배경 노이즈: 6000

✅ 필터링 결과 비교:

  [Stringent Filter]
  - 통과: 10 (0.17%)

  [Sensitive Filter]
  - 통과: 10 (0.17%)

📋 진정 변이 상세:
gene  position variant_type  n_alt_molecules  alt_reads  total_reads  vaf_percent  pass_stringent  pass_sensitive
KRAS  25398284         G12C               33       3293        49106     6.705902            True            True
EGFR  55191822        L858R               23       2530        39668     6.377937            True            True
TP53   7577559        R248W               27       2585        46951     5.505740            True            True
KRAS  25398285         G12V               18       1829        37811     4.837217            True            True
TP53   7577548        R248Q               18       1839        44631     4.120454            True   

In [69]:
# ============================================================================
# 📊 성능 지표 계산 및 출력
# ============================================================================

# 1. 진정 변이 데이터와 배경 노이즈 데이터 분리
true_variants = sim_ctdna[sim_ctdna['is_true_variant'] == True]
background_noise = sim_ctdna[sim_ctdna['is_true_variant'] == False]

print("\n" + "=" * 80)
print("✅ NGS 시스템 성능 평가")
print("=" * 80)

# 2. 필터별 민감도 (Sensitivity) 및 위양성 (False Positive) 계산
for filter_name in ['stringent', 'sensitive']:
    print(f"\n--- [{filter_name.capitalize()} Filter] ---")
    
    # 2-1. 민감도 (Sensitivity): 진짜 변이를 얼마나 잘 잡았는가?
    # True Positives (TP) = 진짜 변이 중 필터를 통과한 수
    tp_count = true_variants[f'pass_{filter_name}'].sum()
    sensitivity = tp_count / len(true_variants) if len(true_variants) > 0 else 0
    
    print(f"  - 🎯 민감도 (Sensitivity): {sensitivity * 100:.2f}% ({tp_count}/{len(true_variants)} 변이 검출)")
    
    # 2-2. 위양성률 (False Positive Rate): 노이즈를 변이라고 착각한 비율
    # False Positives (FP) = 배경 노이즈 중 필터를 통과한 수
    fp_count = background_noise[f'pass_{filter_name}'].sum()
    fpr = fp_count / len(background_noise) if len(background_noise) > 0 else 0
    
    print(f"  - 👻 위양성률 (FPR): {fpr * 100:.4f}% ({fp_count}/{len(background_noise)} 노이즈 오인)")
    
    # 2-3. 가장 잡기 어려운 변이 (low VAF) 검출 여부 확인
    low_vaf_success = true_variants[
        (true_variants['true_vaf'] == 0.005) & (true_variants[f'pass_{filter_name}'])
    ]
    print(f"  - Low VAF (0.5%) 검출: {len(low_vaf_success)}/{len(true_variants[true_variants['true_vaf'] == 0.005])}개")


✅ NGS 시스템 성능 평가

--- [Stringent Filter] ---
  - 🎯 민감도 (Sensitivity): 90.91% (10/11 변이 검출)
  - 👻 위양성률 (FPR): 0.0000% (0/6000 노이즈 오인)
  - Low VAF (0.5%) 검출: 1/2개

--- [Sensitive Filter] ---
  - 🎯 민감도 (Sensitivity): 90.91% (10/11 변이 검출)
  - 👻 위양성률 (FPR): 0.0000% (0/6000 노이즈 오인)
  - Low VAF (0.5%) 검출: 1/2개
